In [1]:
import abc
from dataclasses import dataclass, field
import json
from pathlib import Path
from typing import *
import sys

from beartype import beartype
import datasets
import numpy as np
import more_itertools
import queue
import rich
import torch
import torch.nn as nn
import transformers
import wandb

TokenizerType = transformers.tokenization_utils_fast.PreTrainedTokenizerFast

class BaseEpsilonScheduler(abc.ABC):
    @abc.abstractmethod
    def __call__(self):
        pass

class LinearEpsilonScheduler(BaseEpsilonScheduler):
    def __init__(self, epsilon, num_steps):
        self.epsilon = epsilon
        self.num_steps = num_steps
        self.epoch = 0

    def __call__(self):
        self.epoch += 1
        epsilon = min(self.epsilon * (1 - self.epoch / self.num_epochs), 1)
        wandb.log({"epsilon": epsilon})
        wandb.log({"epsilon_num_steps": self.num_steps})
        return epsilon

class ConstantEpsilonScheduler(BaseEpsilonScheduler):
    def __init__(self, epsilon):
        self.epsilon = epsilon

    def __call__(self):
        epsilon = self.epsilon
        wandb.log({"epsilon": epsilon})
        return epsilon

In [2]:
class BaseRetriever(abc.ABC):
    @abc.abstractmethod
    def retrieve(self, query_ids, query_index):
        pass


class StupidRetriever(BaseRetriever): 
    @beartype
    def __init__(
        self, *, model, tokenizer: TokenizerType, device: Union[int, str], 
        train_vectors: torch.Tensor, train_samples: List[str], train_labels: List[int]
    ):
    
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        self.train_vectors = train_vectors
        self.train_samples = train_samples
        self.train_labels = train_labels
        self.classification_ids_to_idx = {}

        for i, sample in enumerate(train_samples):
            encoded = tokenizer.encode(sample, truncation=True, padding=True, return_tensors="pt")
            self.classification_ids_to_idx[encoded] = i

    def retrieve(self, query_ids, query_index):
        # Get the representation
        representation = self.train_vectors[query_index]
        with torch.inference_mode():
            # Compute the inner products
            scores = torch.matmul(representation, self.train_vectors.t())
            # Get the top 2 results, to potentially exclude the sample itself.
            topk = torch.topk(scores, k=2, dim=-1)
        topk = topk.indices.cpu().numpy()
        
        for retrieved_idx in topk:
            if retrieved_idx != query_index:
                return self.train_samples[retrieved_idx], self.train_labels[retrieved_idx], retrieved_idx
        

# build train vectors
@beartype
def make_retrival_model_and_vectors(
    retriever_name: str, path_to_vectors: Union[str, Path], device: int, dataset_name: str
):
    """We expect the dir to have the following structure:
    - config.json
    - train_samples.json 
    - train_vectors.npy
    """    
    # Make some checks
    config =  json.loads((path_to_vectors / "config.json").read_text())
    assert dataset_name == config["dataset_name"], (dataset_name, config["dataset_name"])
    assert retriever_name == config["retriever_name"], (retriever_name, config["retriever_name"])

    retriever_model = transformers.AutoModel.from_pretrained(retriever_name)
    retriever_tokenizer = transformers.AutoTokenizer.from_pretrained(retriever_name)

    with open(path_to_vectors / "train_samples.json") as f:
        train_samples = json.load(f)
        
    vectors = torch.tensor(np.load(path_to_vectors / "train_vectors.npy")).to(device)
    retriever = StupidRetriever(
        model=retriever_model, 
        tokenizer=retriever_tokenizer, 
        device=device, 
        train_vectors=vectors, 
        train_samples=train_samples["inputs"],
        train_labels=train_samples["labels"],
    )
    
    return retriever


@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)


class BoostingIterator(torch.utils.data.IterableDataset):
    @beartype
    def __init__(
        self, 
        *args, 
        dataset, 
        retriever_client: BaseRetriever, 
        classifier: nn.Module, seed: int, 
        classification_device: Union[int, str], 
        classification_tokenizer: TokenizerType, 
        retriever_device: Union[int, str],
        epsilon_scheduler: BaseEpsilonScheduler, 
        loss_ema_alpha: float, 
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        self.dataset = dataset.map(
            lambda example, idx:{"index": idx}, with_indices=True
        ).shuffle(seed=seed)
        self.priority_queue = queue.PriorityQueue()
        self.retriever_client = retriever_client
        self.epsilon_scheduler = epsilon_scheduler
        self.randomizer = np.random.RandomState(seed)
        self.seed = seed
        self.dataset_iter = None
        self.classifier = classifier
        self.classification_tokenizer = classification_tokenizer
        self.classification_device = classification_device
        self.retriever_device = retriever_device
        self.loss_moving_average = None
        self.loss_ema_alpha = loss_ema_alpha

        # assert mode in ["epsilon_priority_no_reset", "pure_sampled", "epsilon_sampled"], mode

    def push_score(self, inputs, loss):
        average_loss = loss.mean()
        if self.loss_moving_average is None:
            self.loss_moving_average = average_loss
        else:
            self.loss_moving_average = (
                self.loss_ema_alpha * self.loss_moving_average + (1 - self.loss_ema_alpha) * average_loss
            )

        for input_, mask, loss_, index in (
            more_itertools.zip_equal(inputs["input_ids"], inputs["attention_mask"], loss, inputs["index"])
        ):
            assert loss_.shape == torch.Size([]), loss_.shape
            self.priority_queue.put(
                PrioritizedItem(
                    priority= -loss_.detach().cpu().numpy() / self.loss_moving_average, 
                    item=dict(input_ids=input_, attention_mask=mask, index=index)
                    )
                )

    def __len__(self):
        return len(self.dataset)

    def __iter__(self):
        rich.print("[bold green]ITER[/]")
        self.dataset = self.dataset.shuffle(seed=self.seed)
        self.dataset_iter = iter(self.dataset)
        return self
    
    def __next__(self):
        """ This is where the sampling happens.
        """

        # Test if we have a sample and if we pass the epsilon threshold
        empty = self.priority_queue.empty()
        rand = self.randomizer.rand()
        if not empty and rand < self.epsilon_scheduler():
            # pull a sample from the priority queue
            sample = self.priority_queue.get().item

            # We retrieve the next sample.
            input_, next_label, index = self.retriever_client.retrieve(
                sample["input_ids"], sample["index"]
            )
            next_sample = dict(text=input_, label=next_label, index=index)
        else:
            next_sample = next(self.dataset_iter)  # We raise here if we have no more samples in the dataset
            assert next_sample.keys() == {"text", "label", "index"}, next_sample.keys()

        tokenized = self.classification_tokenizer.encode_plus(
            next_sample["text"].strip(), 
            truncation=True, 
            padding=True,
        )

        # text is not needed anymore
        del next_sample["text"]
        assert len(tokenized.keys() & next_sample.keys()) == 0, (tokenized.keys(), next_sample.keys()) 
        return dict(**tokenized, **next_sample)


class BoostingTrainer(transformers.Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.
        Subclass and override to inject custom behavior.
        Args:
            model (`nn.Module`):
                The model to train.
            inputs (`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
                The dictionary will be unpacked before being fed to the model. 
                Most models expect the targets under the
                argument `labels`. Check your model's documentation for all accepted arguments.
        Return:
            `torch.Tensor`: The tensor with training loss on this batch.
        """
        model.train()
        inputs = self._prepare_inputs(inputs)
        index = inputs["index"]
        # Compute loss doesn't work with extra arguments.
        del inputs["index"]

        with self.autocast_smart_context_manager():
            # Get the loss
            loss, outputs = self.compute_loss(model, inputs, return_outputs=True)

        if self.args.n_gpu > 1:
            # Mean over per gpu averages
            loss = loss.mean()  # mean() to average on multi-gpu parallel training

        # This is ignored in the priority queue computation
        if self.args.gradient_accumulation_steps > 1 and not self.deepspeed:
            # Deepspeed handles loss scaling by gradient_accumulation_steps in its `backward`
            loss = loss / self.args.gradient_accumulation_steps
        if self.do_grad_scaling:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with torch.cuda.amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            # loss gets scaled under gradient_accumulation_steps in deepspeed
            loss = self.deepspeed.backward(loss)
        else:
            loss.backward()

        loss = loss.detach()

        # Addition for RetroBoost
        # Make sure the losses are similar, then push them to the priority queue
        # Put index back in

        inputs["index"] = index

        computed_loss = torch.nn.functional.cross_entropy(outputs.logits.detach(), inputs["labels"].detach(), reduction="mean")
        loss_per_sample = torch.nn.functional.cross_entropy(outputs.logits.detach(), inputs["labels"].detach(), reduction="none")
        assert torch.allclose(loss, computed_loss, atol=0.1)
        self.get_train_dataloader().dataset.push_score(inputs, loss_per_sample)

        return loss




/home/mila/g/gagnonju/.main/lib/python3.9/site-packages/beartype/_util/hint/pep/utilpeptest.py:396: BeartypeDecorHintPepDeprecatedWarning: Type hint typing.List[str] deprecated by PEP 585. To resolve this, globally replace this hint by the equivalent PEP 585 type hint (e.g., "typing.List[int]" by "list[int]"). See also:
    https://www.python.org/dev/peps/pep-0585
  warn(warning_message, BeartypeDecorHintPepDeprecatedWarning)
/home/mila/g/gagnonju/.main/lib/python3.9/site-packages/beartype/_util/hint/pep/utilpeptest.py:396: BeartypeDecorHintPepDeprecatedWarning: Type hint typing.List[int] deprecated by PEP 585. To resolve this, globally replace this hint by the equivalent PEP 585 type hint (e.g., "typing.List[int]" by "list[int]"). See also:
    https://www.python.org/dev/peps/pep-0585
  warn(warning_message, BeartypeDecorHintPepDeprecatedWarning)


In [3]:
RETRIEVER_NAME = "facebook/contriever"
DATASET_NAME = "ag_news"
PATH_TO_VECTORS = Path(f"./vectors_{DATASET_NAME}_{RETRIEVER_NAME.split('/')[-1]}/")
CLASSIFIER_NAME = "roberta-base"
CLASSIFIER_BATCH_SIZE = 40
EPSILON_SCHEDULER_TYPE = "constant"
EPSILON_SCHEDULER_CONFIG = dict(
    epsilon=0.5,
)
LOSS_EMA_ALPHA = 0.5
REGULAR_TRAINER = True

CLASSIFIER_DEVICE = 1
RETRIEVER_DEVICE = 2
SEED = 0
SPLIT_RATIO = 0.85

###############################################################################
# Fast setup 
###############################################################################
config = dict(
        classifier_batch_size=CLASSIFIER_BATCH_SIZE,
        classifier_name=CLASSIFIER_NAME,
        dataset_name=DATASET_NAME,
        epsilon=dict(
            scheduler_type=EPSILON_SCHEDULER_TYPE,
            scheduler_config=EPSILON_SCHEDULER_CONFIG,
        ),
        loss_ema_alpha=LOSS_EMA_ALPHA,
        random_seed=SEED,
        regular_trainer=REGULAR_TRAINER,
        retriever_name=RETRIEVER_NAME,
        split_ratio=SPLIT_RATIO,
    )

wandb.init(
    config=config,
    project="RetroBoost", 
    entity="julesgm",
    name="baseline-trainer",
)

EPSILON_SCHEDULER_TYPE_MAP = dict(
    constant=ConstantEpsilonScheduler,
)

# Random seeds. 
np.random.seed(0)
torch.manual_seed(0)

dataset = datasets.load_dataset(DATASET_NAME)
ALL_LABELS = set(dataset["train"]["label"])
NUM_LABELS = len(ALL_LABELS)
assert ALL_LABELS == set(range(NUM_LABELS))

classifier_name = CLASSIFIER_NAME
dataset_name = DATASET_NAME
regular_trainer = REGULAR_TRAINER


classifier = transformers.AutoModelForSequenceClassification.from_pretrained(
    classifier_name, num_labels=NUM_LABELS
)
classifier_tokenizer = transformers.AutoTokenizer.from_pretrained(classifier_name)

def preprocess_function(examples, tokenizer):
    return tokenizer(examples["text"], truncation=True, padding=True)

def preprocess_function(examples, tokenizer):
    return tokenizer(examples["text"], truncation=True, padding=True)


dataset_train = datasets.load_dataset(DATASET_NAME, split=f"train[:{SPLIT_RATIO:.0%}]")
dataset_validation = datasets.load_dataset(DATASET_NAME, split=f"train[{SPLIT_RATIO:.0%}:]")

tokenized_training = dataset_train.map(
    lambda examples: preprocess_function(examples, classifier_tokenizer), 
    batched=True
).shuffle(seed=SEED)

tokenized_validation = dataset_validation.map(
    lambda examples: preprocess_function(examples, classifier_tokenizer), 
    batched=True
).shuffle(seed=SEED)

training_args = transformers.TrainingArguments(
    eval_steps=500,
    evaluation_strategy="steps",
    output_dir="./results",
    learning_rate=1e-5,
    per_device_train_batch_size=CLASSIFIER_BATCH_SIZE,
    per_device_eval_batch_size=int(CLASSIFIER_BATCH_SIZE * 1.5),
    num_train_epochs=10,
    report_to="wandb",
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: retroboost (use `wandb login --relogin` to force relogin)


Using custom data configuration default
Reusing dataset ag_news (/home/mila/g/gagnonju/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?ba/s]

In [4]:
retriever = make_retrival_model_and_vectors(
    retriever_name=RETRIEVER_NAME, 
    path_to_vectors=PATH_TO_VECTORS, 
    device=RETRIEVER_DEVICE, 
    dataset_name=DATASET_NAME,
)
retriever_client = retriever

if regular_trainer:
    trainer = transformers.Trainer(
        model=classifier.to(CLASSIFIER_DEVICE), 
        args=training_args, 
        tokenizer=classifier_tokenizer, 
        train_dataset=tokenized_training, 
        eval_dataset=tokenized_validation,
        data_collator=transformers.DataCollatorWithPadding(
            tokenizer=classifier_tokenizer
        ),
    )
else:
    tokenized_training = BoostingIterator(
        dataset=dataset["train"], 
        retriever_client=retriever_client, 
        classifier=classifier, 
        epsilon_scheduler=EPSILON_SCHEDULER_TYPE_MAP[EPSILON_SCHEDULER_TYPE](**EPSILON_SCHEDULER_CONFIG), 
        seed=SEED,
        retriever_device=RETRIEVER_DEVICE, 
        classification_device=CLASSIFIER_DEVICE,
        classification_tokenizer=classifier_tokenizer,
        loss_ema_alpha=LOSS_EMA_ALPHA,
    )
    
    trainer = BoostingTrainer(
        model=classifier.to(CLASSIFIER_DEVICE),
        args=training_args, 
        tokenizer=classifier_tokenizer, 
        train_dataset=tokenized_training, 
        eval_dataset=tokenized_validation,
        data_collator=transformers.DataCollatorWithPadding(
            tokenizer=classifier_tokenizer
        ),
    )

In [5]:
output = trainer.train()
wandb.finish()

/home/mila/g/gagnonju/.main/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4
  Num Epochs = 10
  Instantaneous batch size per device = 40
  Total train batch size (w. parallel, distributed & accumulation) = 120
  Gradient Accumulation steps = 1
  Total optimization steps = 10
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


KeyError: 0

In [ ]:
retriever

[datasets.arrow_dataset.Dataset,
 datasets.arrow_dataset.DatasetInfoMixin,
 datasets.search.IndexableMixin,
 datasets.arrow_dataset.TensorflowDatasetMixin,
 object]

In [ ]:
dir(bt.meta)

['AUTHORS',
 'AUTHOR_EMAIL',
 'COPYRIGHT',
 'LIBS_DEVELOPER_MANDATORY',
 'LIBS_DOCTIME_MANDATORY',
 'LIBS_DOCTIME_MANDATORY_RTD',
 'LIBS_RUNTIME_OPTIONAL',
 'LIBS_TESTTIME_MANDATORY',
 'LIBS_TESTTIME_MANDATORY_COVERAGE',
 'LIBS_TESTTIME_MANDATORY_TOX',
 'LIBS_TESTTIME_OPTIONAL',
 'LICENSE',
 'NAME',
 'PACKAGE_NAME',
 'PYTHON_VERSION_MIN',
 'PYTHON_VERSION_MINOR_MAX',
 'PYTHON_VERSION_MIN_PARTS',
 'SYNOPSIS',
 'URL_DOWNLOAD',
 'URL_HOMEPAGE',
 'URL_ISSUES',
 'URL_RELEASES',
 'VERSION',
 'VERSION_PARTS',
 '_LIB_RUNTIME_OPTIONAL_VERSION_MINIMUM_NUMPY',
 '_LIB_RUNTIME_OPTIONAL_VERSION_MINIMUM_TYPING_EXTENSIONS',
 '_LIB_VERSION_MIN_SPHINX',
 '_LIB_VERSION_MIN_SPHINX_RTD_THEME',
 '_PYTHON_VERSION_PARTS',
 '_Tuple',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_convert_version_str_to_tuple',
 '_is_os_macos',
 '_is_py_pypy',
 '_sys']

In [24]:
# tokenized_training = dataset_tok[:int(SPLIT_RATIO * len(dataset_tok))]
# tokenized_validation = dataset_tok[int(SPLIT_RATIO * len(dataset_tok)):

Reusing dataset imdb (/home/mila/g/gagnonju/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Reusing dataset imdb (/home/mila/g/gagnonju/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


In [21]:
train[-1]

{'text': 'I really can say I felt the movie in its right essence where the mind games from dreamy reality enter into the surreal aspect of future faced by Tom Cruise. I didn\'t cared much about Tom Cruise\'s acting prowess but I must say that he seems to impress at every point in the movie...not simply due to an engaging storyline but also due to his self being imparted to the lead character....they merge and then speak and its beautiful. However I must say this movie doesn\'t come under the "average flick of weekend" which you pick at random and watch gleefully; It carries strong sentiments and characters so don\'t wash this one down with your beer and pop-corns. It certainly needs more than that.',
 'label': 1}

KeyError: "Column test not in the dataset. Current columns in the dataset: ['text', 'label']"